# TorchIP: Lennard-Jones potential
An example notebook that shows how to reconstruct a Lennard-Jones potential using high-dimensional neural network potential (HDNNP). 


TODOs
- [ ] GPU: implemtation 
- [ ] optimization: multi-thread/process
- [ ] Improve training algorithm

In [ ]:
!gpustat

### Imports

In [ ]:
import sys
sys.path.append('../')

import torchip as tp
from torchip import logger
from torchip.structure import Structure
from torchip.datasets import RunnerStructureDataset, ToStructure
from torchip.utils import gradient, get_value, set_as_attribute
from torchip.descriptors import ASF, CutoffFunction, G2
from torchip.descriptors import DescriptorScaler
from torchip.potentials import NeuralNetworkPotential

import torch
import logging
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
from scipy.optimize import curve_fit
import matplotlib.pylab as plt
from torch.utils.data import DataLoader

In [ ]:
np.random.seed(2022)
torch.manual_seed(2022);
# torch.multiprocessing.set_start_method('spawn')

# logger.set_level(logging.DEBUG)

# print(tp.__doc__)
# print(f"TorchIP {tp.__version__}")

# Set device eigher cpu or cuda (gpu)
tp.device.DEVICE = "cpu"

In [ ]:
potdir = Path('./LJ')

### Strcture dataset

In [ ]:
structures = RunnerStructureDataset(Path(potdir, "input.data"), transform=ToStructure(), persist=False)

### Lennard-Jones energy and force

In [ ]:
# def potential(r, esp=1.0, sig=1.0):
#     """
#     Explicit definition of the Lennard-Jones potential (for debugging). 
#     """
#     tmp = (sig/r)**6
#     return 4.0 * esp * (tmp**2 - tmp) 

# def force(R, esp=1.0, sig=1.0):
#     """
#     Explicit definition of the Lennard-Jones force vector (for debugging).
#     """
#     R = torch.atleast_2d(R)
#     r = torch.norm(R, dim=1)
#     tmp = (sig/r)**6
#     return 24.0 * esp / r**2 * (2.0*tmp**2 - tmp) * R

In [ ]:
# data = defaultdict(list)
# data_loader = DataLoader(structures, collate_fn=lambda batch: batch)

# for batch in data_loader:
#     structure = batch[0]
#     r = structure.calculate_distance(aid=0, neighbors=1) 
#     E = structure.total_energy #potential(r)
#     F = structure.force #-gradient(E, structure.position)
#     R = structure.position
# #     print("r:", get_value(r))
# #     print("E:", get_value(E))
# #     print("F:", get_value(F))
# #     print("R:", get_value(R))
# #     print()
# #     break

# #     Rij = R[0] - R[1]
# #     print("auto-diff:", get_val(F[0]))
# #     print("trueforce:", get_val(force(Rij)[0]))
# #     print()

#     data['energy'].append(get_value(E))
#     data['distance'].append(get_value(r))

In [ ]:
# plt.scatter(data['distance'], data['energy'])
# plt.xlabel("distance"); plt.ylabel("energy");

##### Finding epsilon and sigma parameters

In [ ]:
# x = np.array(data['distance'])[:, 0]
# y = np.array(data['energy'])[:, 0]

# popt, pcov = curve_fit(potential, x, y)
# print(f"eps = {popt[0]}, sig = {popt[1]}")

## Potential

In [ ]:
# Pptential
pot = NeuralNetworkPotential(Path(potdir, "input.nn"))

### Descriptors

In [ ]:
# # Cutoff function
# r_cutoff, cutoff_type = 3.0, "tanh"
# cfn = CutoffFunction(r_cutoff, cutoff_type)

# # Descriptor
# asf = ASF("Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.00), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.25), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.50), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.75), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=1.00), "Ne")

# from torchip.utils.profiler import torch_profile
# @torch_profile
# def kernel(structures):
#     return pot.fit_scaler(structures)
# kernel(structures)

pot.load_scaler()
# pot.fit_scaler(structures)

### Model

#### Training

In [ ]:
# logger.set_level(logging.DEBUG)

In [ ]:
%time history = pot.fit_model(structures, epochs=100, validation_split=0.20)
# pot.load_model()
# history = profile(pot.fit_model, structures, epochs=0, validation_split=0.20)

In [ ]:
df = pd.DataFrame(history)
df[["train_energy_rmse", "valid_energy_rmse"]][:].plot();
df[["train_force_rmse", "valid_force_rmse"]][:].plot();

In [ ]:
df.tail()

#### Energy & Forces

In [ ]:
def mse(predictions, targets):
    if predictions.ndim > 1:
        return ((predictions - targets) ** 2).mean(axis=0)
    return ((predictions - targets) ** 2).mean()

def rmse(predictions, targets):
    return np.sqrt(mse(predictions, targets))

df = defaultdict(list)
for structure in structures:
    energy = pot(structure)
    E_pred = get_value(energy)
    E_true = get_value(structure.total_energy)   
    df['E_pred'].append(E_pred[0])
    df['E_true'].append(E_true[0])
    df['E_mse'].append(mse(E_pred, E_true)) 
    df['E_rmse'].append(rmse(E_pred, E_true))
    df['E_err'].append((E_true - E_pred)[0])
    
    force = -gradient(energy, structure.position)
    F_pred = get_value(force)
    F_true = get_value(structure.force)
    
    df['F_pred'].append(F_pred[0][0])
    df['F_true'].append(F_true[0][0])
    df['F_mse'].append(mse(F_pred, F_true)[0]) 
    df['F_rmse'].append(rmse(F_pred, F_true)[0])
    df['F_err'].append((F_true - F_pred)[0][0])
    
    # print("Predicted energy:\n", E_pred)
    # print("True energy:\n", E_true)
    # print("MSE:\n", mse(E_pred, E_true))
    # print("RMSE:\n", rmse(E_pred, E_true))
    # print("Predicted force: \n", F_pred )
    # print("True force:\n", F_true)
    # print("MSE:\n", mse(F_pred, F_true))
    # print("RMSE:\n", rmse(F_pred, F_true))
    
df = pd.DataFrame(df)
df

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,8))

df.plot(y=['E_pred', 'E_true'], ax=ax[0][0])
# df.plot(y=['E_rmse'], ax=ax[1][0])
df.plot.hist(y=['E_err'], ax=ax[1][0])

df.plot(y=['F_pred', 'F_true'], ax=ax[0][1])
# df.plot(y=['F_rmse'], ax=ax[1][1])
df.plot.hist(y=['F_err'], ax=ax[1][1]);